In [39]:
# from PyPDF2 import PdfReader
# import src.ConvertPdftoTxt as cpt
from tqdm import tqdm
import os
import re
import json

In [40]:
# import pandas as pd
# pd.DataFrame(os.listdir(output_folder))

In [41]:
pdf_folder_path = '/home/minds/army_smart/data/jisi/'
output_folder = '/home/minds/army_smart/data/jisi/'

## pdf -> txt 변환
# cpt.save_all_pdfs_as_texts(pdf_folder_path, output_folder)

In [42]:
## txt file open
# txt_file_name = os.listdir(output_folder)[0] ## file_name
txt_file_name = '22-1012.txt'

txt_list = []
with open(os.path.join(output_folder, txt_file_name),'r') as f:
    line = f.readlines()
    txt_list += line
f.close()

In [43]:
txt_list

['분 류 : 육본지시 제 22-1012호 발령일자 :2022. 7. 22.\n',
 '발 신 :참모총장 유효기간 :2023. 7. 21.\n',
 '수 신 :대대장급 이상 지휘관\n',
 '제 목 :「2022년도 육군 제수당 지급」지시\n',
 '육  군  본  부순        서\n',
 'Ⅰ. 총    칙  · · · · · · · · · · · · · · · · · · · · · · · · ·   1\n',
 '  Ⅱ. 상여수당 \n',
 '      1. 대우군무원 수당  · · · · · · · · · · · · · · · · · · · ·   4\n',
 '      2. 정근수당 · · · · · · · · · · · · · · · · · · · · · · · ·   5\n',
 '      3. 정근수당 가산금  · · · · · · · · · · · · · · · · · · ·  10\n',
 '  Ⅲ. 가계보전수당\n',
 '      1. 가족수당 · · · · · · · · · · · · · · · · · · · · · · · ·  12\n',
 '      2. 재외공무원 자녀학비보조수당 · · · · · · · · · · · · · ·  22\n',
 '      3. 주택수당 · · · · · · · · · · · · · · · · · · · · · · ·  27\n',
 '      4. 육아휴직수당 · · · · · · · · · · · · · · · · · · · · ·  28\n',
 '  Ⅳ. 특수지근무수당  ·  · · · · · · · · · · · · · · · · · · · · 34\n',
 '  Ⅴ. 특수근무수당\n',
 '      1. 위험근무수당 · · · · · · · · · · · · · · · · · · · · ·  36\n',
 '         가. 군인 위험근무수당 · · · · · · · · · · · · · · · ·  36\n',
 '         나. 군무원 위험근무수당 · · · · ·

In [44]:
## 로마자가 있는 경우, 장으로 바꾸고, 아날로그 숫자만 있는 경우 조로 바꾸기
def roman_to_int(s):
    roman_numerals = {'Ⅰ': 1, 'Ⅱ': 2, 'Ⅲ': 3, 'Ⅳ': 4, 'Ⅴ': 5, 'Ⅵ': 6, 'Ⅶ': 7, 'Ⅷ': 8, 'Ⅸ': 9, 'Ⅹ': 10}
    return roman_numerals.get(s, 0)

if len([txt for txt in txt_list if re.match(r'제\s*\d+\s*조(\(|\[)',txt)]) < 1 : 
    transformed_list = []
    for txt in txt_list:
        # 정규식 패턴으로 로마 숫자를 포함하는 행 찾기
        roman_pattern = r'^\s*(Ⅰ|Ⅱ|Ⅲ|Ⅳ|Ⅴ|Ⅵ|Ⅶ|Ⅷ|Ⅸ|Ⅹ)\. .+?$\n'
        match = re.match(roman_pattern, txt)
        if match:
            roman_numeral = match.group(1)
            arabic_numeral = roman_to_int(roman_numeral)
            # 로마 숫자를 "제 {숫자} 장" 형태로 교체
            # 아라빅 숫자를 "제 {숫자} 조" 형태로 교체
            txt = re.sub(r'^(\s*)Ⅰ|Ⅱ|Ⅲ|Ⅳ|Ⅴ|Ⅵ|Ⅶ|Ⅷ|Ⅸ|Ⅹ', fr'제 {arabic_numeral}장', txt)
            txt = re.sub(r'(.+)(- \d+ -)Ⅰ|Ⅱ|Ⅲ|Ⅳ|Ⅴ|Ⅵ|Ⅶ|Ⅷ|Ⅸ|Ⅹ', f'\1제 {arabic_numeral}장', txt)
            txt = re.sub(r'^(\s*)(\d+)\.(.+)\n', r'\1제\2조(\3)\n', txt)
            txt = re.sub(r'.+(- \d+ -)(\d+)\.(.+)\n', r'제\2조(\3)\n', txt)
            transformed_list.append(txt)
        else:
            txt = re.sub(r'^(\s*)(\d+)\.(.+)\n', r'\1제\2조(\3)\n', txt)
            txt = re.sub(r'.+(- \d+ -)(\d+)\.(.+)\n', r'제\2조(\3)\n', txt)
            transformed_list.append(txt)
            
    txt_list = transformed_list

In [45]:
re.sub(r'(.+)(- \d+ -)Ⅰ|Ⅱ|Ⅲ|Ⅳ|Ⅴ|Ⅵ|Ⅶ|Ⅷ|Ⅸ|Ⅹ', f'\1제 {arabic_numeral}장', '[27(휴직일 )/31]- 34 -Ⅳ. 특수지근무수당 (')

'[27(휴직일 )/31]- 34 -\x01제 8장. 특수지근무수당 ('

In [46]:
txt_list

['분 류 : 육본지시 제 22-1012호 발령일자 :2022. 7. 22.\n',
 '발 신 :참모총장 유효기간 :2023. 7. 21.\n',
 '수 신 :대대장급 이상 지휘관\n',
 '제 목 :「2022년도 육군 제수당 지급」지시\n',
 '육  군  본  부순        서\n',
 '제 1장. 총    칙  · · · · · · · · · · · · · · · · · · · · · · · · ·   1\n',
 '  제 2장. 상여수당 \n',
 '      제1조( 대우군무원 수당  · · · · · · · · · · · · · · · · · · · ·   4)\n',
 '      제2조( 정근수당 · · · · · · · · · · · · · · · · · · · · · · · ·   5)\n',
 '      제3조( 정근수당 가산금  · · · · · · · · · · · · · · · · · · ·  10)\n',
 '  제 3장. 가계보전수당\n',
 '      제1조( 가족수당 · · · · · · · · · · · · · · · · · · · · · · · ·  12)\n',
 '      제2조( 재외공무원 자녀학비보조수당 · · · · · · · · · · · · · ·  22)\n',
 '      제3조( 주택수당 · · · · · · · · · · · · · · · · · · · · · · ·  27)\n',
 '      제4조( 육아휴직수당 · · · · · · · · · · · · · · · · · · · · ·  28)\n',
 '  제 4장. 특수지근무수당  ·  · · · · · · · · · · · · · · · · · · · · 34\n',
 '  제 5장. 특수근무수당\n',
 '      제1조( 위험근무수당 · · · · · · · · · · · · · · · · · · · · ·  36)\n',
 '         가. 군인 위험근무수당 · · · · · · · · · · · · · · · ·  36

In [47]:
## 목차를 지우고 내용만 추출
## 조항 내용 / 별표와 별지 따로 리스트 생성
if len([i for i, txt in enumerate(txt_list) if re.search('제\s*1\s*편', txt)]) >= 2:
    jeoul_idx = [i for i, txt in enumerate(txt_list) if re.search('제 1\s*편', txt)]
    start_raw = jeoul_idx[1]
elif len([i for i, txt in enumerate(txt_list) if re.search('(제\s*1\s*장)', txt)]) >=2 :
    jeoul_idx = [i for i, txt in enumerate(txt_list) if re.search('(제\s*1\s*장)', txt)]
    start_raw = jeoul_idx[1] # 특별건설기술심의위원회 운영 및 시공 등 평가훈령 개정안 전문.txt 일 때는 0으로 변경
else:
    jeoul_idx = [i for i, txt in enumerate(txt_list) if re.search('(제\s*1\s*조)', txt)]
    start_raw = jeoul_idx[1]

## 별표 및 별지 별도 추출
## 별표 및 별지가 없을 경우, 해당 텍스트 자체가 모두 raw_txt
# if len([i for i, txt in enumerate(txt_list) if re.search('(별표\s*및\s*별지)|(별표\s*목차)|(별지\s*목차)|(별표\s*및)|(별표\s*\/\s*서식)', txt)]) >= 2:
#     start_pyo = [i for i, txt in enumerate(txt_list) if re.search('(별표\s*및\s*별지)|(별표\s*목차)|(별지\s*목차)|(별표\s*및)|(별표\s*\/\s*서식)', txt)]
#     start_pyo = max(start_pyo)
#     pyo_contents = [i for i, txt in enumerate(txt_list) if re.search('\〔별지|\[별지|\【별지\〔별표|\[별표|\【별표|\〔별표\s*제?1호?\〕|\[별표\s*제?1호?\]|\【별표\s*제?1호?\】', txt)]

#     raw_txt = txt_list[start_raw:start_pyo]
#     pyo_txt = txt_list[max(pyo_contents):]

# elif len([i for i, txt in enumerate(txt_list) if not re.search('(별표\s*및\s*별지)|(별표\s*목차)|(별지\s*목차)|(별표\s*및)|(별표\s*\/\s*서식)', txt)]) < 1:
#     start_kook = [i for i, txt in enumerate(txt_list) if re.search('(육\s*군\s*참\s*모\s*총\s*장)(국\s*방\s*부\s*장\s*관)', txt)]
#     start_kook = max(start_kook)
#     pyo_contents = [i for i, txt in enumerate(txt_list) if re.search('\〔별지|\[별지|\【별지\〔별표|\[별표|\【별표|\〔별표\s*제?1호?\〕|\[별표\s*제?1호?\]|\【별표\s*제?1호?\】', txt)]

#     raw_txt = txt_list[start_raw:start_kook]
#     pyo_txt = txt_list[max(pyo_contents):]
# else:
#     raw_txt = txt_list[start_raw:]
#     pyo_txt = []

## 별표 및 별지 별도 추출
## 별표 및 별지가 없을 경우, 해당 텍스트 자체가 모두 raw_txt
if len([i for i, txt in enumerate(txt_list) if re.search('(별표\s*및\s*별지)|(별표\s*목차)|(별지\s*목차)|(별표\s*및)|(별표\s*\/\s*서식)', txt)]) >= 2:
    start_pyo = [i for i, txt in enumerate(txt_list) if re.search('(별표\s*및\s*별지)|(별표\s*목차)|(별지\s*목차)|(별표\s*및)|(별표\s*\/\s*서식)', txt)]
    start_pyo = max(start_pyo)
    pyo_contents = [i for i, txt in enumerate(txt_list) if re.search('\〔별지|\[별지|\【별지|\〔별표|\[별표|\【별표|\〔별표\s*제?1호?\〕|\[별표\s*제?1호?\]|\【별표\s*제?1호?\】', txt)]

    raw_txt = txt_list[start_raw:start_pyo]
    pyo_txt = txt_list[max(pyo_contents):]

elif len([i for i, txt in enumerate(txt_list) if re.search('(별표\s*및\s*별지)|(별표\s*목차)|(별지\s*목차)|(별표\s*및)|(별표\s*\/\s*서식)', txt)]) < 1:
    start_kook = [i for i, txt in enumerate(txt_list) if re.search('(육\s*군\s*참\s*모\s*총\s*장)|(국\s*방\s*부\s*장\s*관)', txt)]
    start_kook = max(start_kook)
    pyo_contents = [i for i, txt in enumerate(txt_list) if re.search('\〔별지|\[별지|\【별지|\〔별표|\[별표|\【별표|\〔별표\s*제?1호?\〕|\[별표\s*제?1호?\]|\【별표\s*제?1호?\】', txt)]

    raw_txt = txt_list[start_raw:start_kook]
    pyo_txt = txt_list[max(pyo_contents):]
else:
    raw_txt = txt_list[start_raw:]
    pyo_txt = []

In [48]:
raw_txt

['제 1장. 총 칙\n',
 '제1조(목  적)\n',
 '이 지시는 「공무원 수당 등에 관한 규정」(이하 “영”이라 한다)「군인 등의 특수근무\n',
 '수당에 관한 규칙」(이하 “규칙”이라 한다)및「공무원 보수 등의 업무지침 」등에 의하여\n',
 '군인 및 군무원에게 적용하는 제수당의 지급기준과 세부절차를 정함을 목적으로 \n',
 '한다.\n',
 '제2조(일반지침)\n',
 '가.수당지급에 관한 사항은 본 지시에 의하고 ,임의로 지급대상자 선정기준 및\n',
 '지급액 등을 조정할 수 없다.\n',
 '나.수당의 지급방법 (중앙지급 ,부대지급 )과 정․결산에 관한 사항은 국군재정관리단장이\n',
 '정한다 .\n',
 '       다. 제수당 신청절차 및 지급\n',
 '수당신청( 개인)• 제수당 지급지침 확인\n',
 '• 제수당 신청( 국방인사정보체계)\n',
 '수당 / 행정담당\n',
 '(부대(서))• 제수당 신청 접수 / 확인\n',
 '• 제수당 지급지침 확인\n',
 '• 제수당 지급 심사위원회 개최 / 선발\n',
 '제수당 \n',
 '지급심사위원회\n',
 '(사(여)단급 이상 부대)• 제수당 지급지시 확인\n',
 '• 제수당 지급 심사위원회 개최 / 선발\n',
 '  - 인사명령 발령\n',
 '  - 국방통합급여정보체계 입력\n',
 '국군재정관리단• 통합급여정보체계 확인\n',
 '• 제수당 입금\n',
 '  - 매월 10일( 실적수당 25일)\n',
 '수당 입금확인( 개인)• 국방통합급여정보체계 확인\n',
 '  - 국방통합급여포탈 → 수당조회- 2 -      라.여단급 (준장)이상 부대의 인사참모는 필요시 분기 단위 심의를 실시한 후 그 결과를\n',
 '재정참모에게 통보한다. 여단 및 대대 인사과장은 수당 심의에 필요한 자료\n',
 '(명령지, 자격증 사본 등)를 국방인사정보체계를 이용하여 여단급(준장)이상 \n',
 '부대의 인사참모에게 제출하여야 한다.\n',
 '(수당 변동사항 발생 시 수시 심의 후

In [49]:
## 편, 장, 절, 조 단위로 자르기
raw_txt = [txt for txt in raw_txt if txt!='\n']

raw = []
for txt in raw_txt:
    if re.match('(\s*)(제\s*[0-9]{1,2}\s*편|제\s*[0-9]{1,2}\s*장|제\s*[0-9]{1,2}\s*절|제\s*[0-9]{1,2}\s*관|제\s*[0-9]{1,3}\s*조\(|제\s*[0-9]{1,3}\s*조\[|부\s*칙|(작\s*성\s*관))', txt):
        split_text = re.split(r'(제\d+조\(|제\d+조\[)', txt)  ## 편/장/절 중간에 조항이 있으면 쪼개기
        if len(split_text) >= 2:
            split_idx = [i for i, s in enumerate(split_text) if re.search(r'(제\d+조\(|제\d+조\[)',s)]
            for idx in sorted(split_idx, reverse=True):
                split_text[idx+1] = split_text[idx] + split_text[idx+1]
                del split_text[idx]
            split_text = [s for s in split_text if s]
            raw.extend(split_text)
        else:
            raw.append(txt)
    else:
        split_text = re.split(r'(제\d+조\(|제\d+조\[)', txt)  ## 문장 중간에 조항이 있으면 쪼개기
        if len(split_text) >= 2:
            split_idx = [i for i, s in enumerate(split_text) if re.search(r'(제\d+조\(|제\d+조\[)',s)]
            for idx in sorted(split_idx, reverse=True):
                split_text[idx+1] = split_text[idx] + split_text[idx+1]
                del split_text[idx]
            split_text = [s for s in split_text if s]
            # raw 가 0일 경우
            if len(raw)==0:
                raw.extend(split_text)
            else:
                raw[-1] = "".join([raw[-1],split_text[0]])
                raw.extend(split_text[1:])
        elif len(raw)==0:
            raw.append(txt)
        else:
            raw[-1] = "".join([raw[-1],txt])


In [50]:
print(raw[6])

제1조(대우군무원 수당(영 제6조의 2))
가.지급기준
대          상 기  준  액 비           고
4급 이하(「군무원인사법
시행령」제44조에 의하여
대우군무원으로 선발된 자,
예비전력군무원 포함)월봉급액의 4.1%
<개정 2011.1.10>◦성과급적 연봉제 적용 대상은 
연봉월액×78%(성과급적 연봉제 적용 
대상 공무원 중기본연봉에 관리업무
수당에 해당하는 금액이 포함되지 않은 
공무원은 84%) x4.1%
◦제한사항
·가족수당 비고란 제한사항과 동일
※다만 ,대우군무원수당과 월봉급액을 합산한 금액이 상위직급으로 승진 시의 월
봉급액을 초과할 경우에는 해당 직급 월 봉급액과 상위 직급 월봉급액의 차액을  
대우군무원수당으로 지급한다 .<인사혁신처 예규 ｢
 2022년 공무원보수 등의 
업무지침｣
나.감액 지급
정직․감봉으로 보수가 감액 지급되는 군무원은 ｢
 군무원인사법｣
제39조에 따라 ,
직위해제 및 휴직으로 봉급이 감액 지급되는 군무원에게는 영 별표 4의 지급
구분에 따라 처분 기간 중 대우군무원수당 (가산금 포함 )을 감액하여 지급한다 .
예 시
ㅇ 정직 3개월 처분 시 감액 계산방법
’22.3.20. ’22.4.1 ’22.5.1 ’22.6.1 ’22.6.19 ’22.6.20
정직처분일 정직처분의집행이 끝난 날 복직
① ’22.3월 분 중 감할 금액 :’22.3.20∼3.31까지의 대우군무원수당액을 일할 
계산한 다음 감액률 2/3를 곱함
-감할 금액 : 월 대우군무원수당액 ×12/31×2/3
＊'22.3.1∼3.19까지는일할계산하여정상지급,정상지급액:당해월
대우군무원수당액×19/31
② ’22.4월분 중 감할 금액 :해당 월 대우군무원수당액 × 2/3
③ ’22.5월분 중 감할 금액 :해당 월 대우군무원수당액 × 2/3
④ ’22.6월분 중 감할 금액 : ’21.6.1∼6.19까지는 일할 계산한 다음 감액률 
2/3를 곱함
-감할 금액 : 해당 월 대우군무원수당액 ×19/30×2/3
＊'22.6.20∼6.30까지는일할계산하여정상지

In [51]:
raw

['제 1장. 총 칙\n',
 '제1조(목  적)\n이 지시는 「공무원 수당 등에 관한 규정」(이하 “영”이라 한다)「군인 등의 특수근무\n수당에 관한 규칙」(이하 “규칙”이라 한다)및「공무원 보수 등의 업무지침 」등에 의하여\n군인 및 군무원에게 적용하는 제수당의 지급기준과 세부절차를 정함을 목적으로 \n한다.\n',
 '제2조(일반지침)\n가.수당지급에 관한 사항은 본 지시에 의하고 ,임의로 지급대상자 선정기준 및\n지급액 등을 조정할 수 없다.\n나.수당의 지급방법 (중앙지급 ,부대지급 )과 정․결산에 관한 사항은 국군재정관리단장이\n정한다 .\n       다. 제수당 신청절차 및 지급\n수당신청( 개인)• 제수당 지급지침 확인\n• 제수당 신청( 국방인사정보체계)\n수당 / 행정담당\n(부대(서))• 제수당 신청 접수 / 확인\n• 제수당 지급지침 확인\n• 제수당 지급 심사위원회 개최 / 선발\n제수당 \n지급심사위원회\n(사(여)단급 이상 부대)• 제수당 지급지시 확인\n• 제수당 지급 심사위원회 개최 / 선발\n  - 인사명령 발령\n  - 국방통합급여정보체계 입력\n국군재정관리단• 통합급여정보체계 확인\n• 제수당 입금\n  - 매월 10일( 실적수당 25일)\n수당 입금확인( 개인)• 국방통합급여정보체계 확인\n  - 국방통합급여포탈 → 수당조회- 2 -      라.여단급 (준장)이상 부대의 인사참모는 필요시 분기 단위 심의를 실시한 후 그 결과를\n재정참모에게 통보한다. 여단 및 대대 인사과장은 수당 심의에 필요한 자료\n(명령지, 자격증 사본 등)를 국방인사정보체계를 이용하여 여단급(준장)이상 \n부대의 인사참모에게 제출하여야 한다.\n(수당 변동사항 발생 시 수시 심의 후 결과를 재정참모에게 통보하며 ,지급대상자 전출 /보직\n변경 시 전 소속부대에서 필수적으로 해임처리 )\n마.재정참모는 심의 결과 중 중앙지급수당과 관련 신규 임명 및 해임 등 변동사항을\n국군재정관리단( 급여연금처 수당관리과) 으로 제출한다.\n바.수당 인사명령 발

In [52]:

# ## 편, 장, 절에 대한 list index 추출
# pyeon_idx = []
# jang_idx = []
# jeoul_idx = []
# gwan_idx = []
# buchik_idx = []
# editor_idx = []

# for i, txt in enumerate(raw):
#     if re.match('.*(제\s*[0-9]{1,3}\s*편)', txt):
#         raw[i] = re.sub(r'.*(제)(\s*[0-9]{1,2}\s*)(편)', r'제 \2편',txt)
#         pyeon_idx.append(i)
#     elif re.match('.*(제\s*[0-9]{1,3}\s*장)', txt):
#         raw[i] = re.sub(r'.*(제)(\s*[0-9]{1,2}\s*)(장)', r'제 \2장',txt)
#         jang_idx.append(i)
#     elif re.match('.*(제\s*[0-9]{1,3}\s*절)', txt):
#         raw[i] = re.sub(r'.*(제)(\s*[0-9]{1,2}\s*)(절)', r'제 \2절',txt)
#         jeoul_idx.append(i)
#     elif re.match('.*(제\s*[0-9]{1,2}\s*관)', txt):
#         raw[i] = re.sub(r'.*(제)(\s*[0-9]{1,2}\s*)(관)', r'제 \2관',txt)
#         gwan_idx.append(i)
#     elif re.match('부\s*칙', txt):
#         buchik_idx.append(i)
#     elif re.match('\s*작\s*성\s*관', txt): # ' 작성관'인 파일도 있어 패턴 \s* 추가
#         editor_idx.append(i)

In [53]:

## 편, 장, 절에 대한 list index 추출
pyeon_idx = []
jang_idx = []
jeoul_idx = []
gwan_idx = []
buchik_idx = []
editor_idx = []

for i, txt in enumerate(raw):
    if re.match('(제\s*[0-9]{1,3}\s*편)', txt):
        raw[i] = re.sub(r'(제)(\s*[0-9]{1,2}\s*)(편)', r'제 \2편',txt)
        pyeon_idx.append(i)
    elif re.match('(제\s*[0-9]{1,3}\s*장)', txt):
        raw[i] = re.sub(r'(제)(\s*[0-9]{1,2}\s*)(장)', r'제 \2장',txt)
        jang_idx.append(i)
    elif re.match('(제\s*[0-9]{1,3}\s*절)', txt):
        raw[i] = re.sub(r'(제)(\s*[0-9]{1,2}\s*)(절)', r'제 \2절',txt)
        jeoul_idx.append(i)
    elif re.match('(제\s*[0-9]{1,2}\s*관)', txt):
        raw[i] = re.sub(r'(제)(\s*[0-9]{1,2}\s*)(관)', r'제 \2관',txt)
        gwan_idx.append(i)
    elif re.match('부\s*칙', txt):
        buchik_idx.append(i)
    elif re.match('\s*작\s*성\s*관', txt): # ' 작성관'인 파일도 있어 패턴 \s* 추가
        editor_idx.append(i)

In [54]:
raw

['제  1장. 총 칙\n',
 '제1조(목  적)\n이 지시는 「공무원 수당 등에 관한 규정」(이하 “영”이라 한다)「군인 등의 특수근무\n수당에 관한 규칙」(이하 “규칙”이라 한다)및「공무원 보수 등의 업무지침 」등에 의하여\n군인 및 군무원에게 적용하는 제수당의 지급기준과 세부절차를 정함을 목적으로 \n한다.\n',
 '제2조(일반지침)\n가.수당지급에 관한 사항은 본 지시에 의하고 ,임의로 지급대상자 선정기준 및\n지급액 등을 조정할 수 없다.\n나.수당의 지급방법 (중앙지급 ,부대지급 )과 정․결산에 관한 사항은 국군재정관리단장이\n정한다 .\n       다. 제수당 신청절차 및 지급\n수당신청( 개인)• 제수당 지급지침 확인\n• 제수당 신청( 국방인사정보체계)\n수당 / 행정담당\n(부대(서))• 제수당 신청 접수 / 확인\n• 제수당 지급지침 확인\n• 제수당 지급 심사위원회 개최 / 선발\n제수당 \n지급심사위원회\n(사(여)단급 이상 부대)• 제수당 지급지시 확인\n• 제수당 지급 심사위원회 개최 / 선발\n  - 인사명령 발령\n  - 국방통합급여정보체계 입력\n국군재정관리단• 통합급여정보체계 확인\n• 제수당 입금\n  - 매월 10일( 실적수당 25일)\n수당 입금확인( 개인)• 국방통합급여정보체계 확인\n  - 국방통합급여포탈 → 수당조회- 2 -      라.여단급 (준장)이상 부대의 인사참모는 필요시 분기 단위 심의를 실시한 후 그 결과를\n재정참모에게 통보한다. 여단 및 대대 인사과장은 수당 심의에 필요한 자료\n(명령지, 자격증 사본 등)를 국방인사정보체계를 이용하여 여단급(준장)이상 \n부대의 인사참모에게 제출하여야 한다.\n(수당 변동사항 발생 시 수시 심의 후 결과를 재정참모에게 통보하며 ,지급대상자 전출 /보직\n변경 시 전 소속부대에서 필수적으로 해임처리 )\n마.재정참모는 심의 결과 중 중앙지급수당과 관련 신규 임명 및 해임 등 변동사항을\n국군재정관리단( 급여연금처 수당관리과) 으로 제출한다.\n바.수당 인사명령 

In [55]:
def gwan_loop(gwan_idx, raw, i): # 절이 있을때 loop
    if len(gwan_idx) > 0:
        for gwan_i, gwan in enumerate(gwan_idx):
            if i > gwan and (gwan_i == len(gwan_idx) - 1 or i < gwan_idx[gwan_i + 1]):
                raw[i] = raw[gwan] + raw[i]
                break
    return raw

def jeoul_loop(jeoul_idx, gwan_idx, raw, i): # 절이 있을때 loop
    if len(jeoul_idx) > 0:
        for jeoul_i, jeoul in enumerate(jeoul_idx):
            if i > jeoul and (jeoul_i == len(jeoul_idx) - 1 or i < jeoul_idx[jeoul_i + 1]):
                if jeoul+1 in gwan_idx:
                    gwan_loop(gwan_idx, raw, i)
                    raw[i] = raw[jeoul] + raw[i]
                    break
                else: #관이 없는 경우
                    raw[i] = raw[jeoul] + raw[i]
                    break
    else:
        raw = gwan_loop(gwan_idx, raw, i)
    return raw

def jang_loop(jang_idx, jeoul_idx, gwan_idx, raw, i):
    if len(jang_idx) > 0:
        for jang_i, jang in enumerate(jang_idx):
            # 현재 인덱스가 jang보다 크고, 다음 jang 인덱스가 현재 인덱스보다 크지 않은 경우
            if i > jang and (jang_i == len(jang_idx) - 1 or i < jang_idx[jang_i + 1]):
                if jang+1 in jeoul_idx:
                    jeoul_loop(jeoul_idx, gwan_idx, raw, i)
                    raw[i] = raw[jang] + raw[i]
                    break
                elif jang+1 in gwan_idx:
                    gwan_loop(gwan_idx, raw, i)
                    raw[i] = raw[jang] + raw[i]
                    break
                else: # 절이 없는 경우
                    raw[i] = raw[jang] + raw[i]
                    break
    else: 
        raw = jeoul_loop(jeoul_idx, gwan_idx, raw, i)
    return raw

def pyeon_loop(pyeon_idx, jang_idx, jeoul_idx, gwan_idx, raw, i): # 편이 있을 때 loop
    if len(pyeon_idx) > 0:
        for pyeon_i, pyeon in enumerate(pyeon_idx):
            if i > pyeon and (pyeon_i == len(pyeon_idx) - 1 or i < pyeon_idx[pyeon_i + 1]):
                if pyeon+1 in jang_idx:
                    jang_loop(jang_idx, jeoul_idx, gwan_idx, raw, i)
                    raw[i] = raw[pyeon] + raw[i]
                    break
                elif pyeon+1 in jeoul_idx:
                    jeoul_loop(jeoul_idx, gwan_idx, raw, i)
                    raw[i] = raw[pyeon] + raw[i]
                    break
                else:
                    raw[i] = raw[pyeon] + raw[i]
    else: 
        raw = jang_loop(jang_idx, jeoul_idx, gwan_idx, raw, i)
    return raw

## 편, 장, 절을 조 앞에 붙이기
for i in tqdm(range(len(raw))):
    flag = False
    if i not in gwan_idx + jang_idx + jeoul_idx + pyeon_idx + buchik_idx:
        try:
            if i <buchik_idx[0]: ## 부칙은 끝쪽에 있으니 그 전에
                raw = pyeon_loop(pyeon_idx, jang_idx, jeoul_idx, gwan_idx, raw, i)
                if re.search('제\s*\d+\s*조', raw[i]) is None:
                    raw[i] = ''
            else:    ## 부칙 있는 부분
                for buchik_i, buchik in enumerate(buchik_idx):
                    if (i > buchik)&(i > buchik_idx[buchik_i + 1]): continue
                    if i > buchik:
                        raw[i] = raw[buchik] + raw[i]
                    else: break
        except IndexError:
            pass
    else: continue


## 편, 장, 절 만 있는 값 삭제
raw = [txt for i, txt in enumerate(raw) if i not in gwan_idx+jang_idx + jeoul_idx + pyeon_idx]
raw = [txt for txt in raw if txt]
raw = [txt for txt in raw if not re.search('(부칙\n$)|[0-9]{4}.\s*[0-9]{1,2}.\s*[0-9]{1,2}.(>|\))\s*\n$',txt)]

100%|██████████| 53/53 [00:00<00:00, 116021.98it/s]


In [92]:
print(raw[1])

IndexError: list index out of range

In [58]:
temp = raw

In [114]:
raw = [temp[1]]
raw

['제2조(일반지침)\n가.수당지급에 관한 사항은 본 지시에 의하고 ,임의로 지급대상자 선정기준 및\n지급액 등을 조정할 수 없다.\n나.수당의 지급방법 (중앙지급 ,부대지급 )과 정․결산에 관한 사항은 국군재정관리단장이\n정한다 .\n       다. 제수당 신청절차 및 지급\n수당신청( 개인)• 제수당 지급지침 확인\n• 제수당 신청( 국방인사정보체계)\n수당 / 행정담당\n(부대(서))• 제수당 신청 접수 / 확인\n• 제수당 지급지침 확인\n• 제수당 지급 심사위원회 개최 / 선발\n제수당 \n지급심사위원회\n(사(여)단급 이상 부대)• 제수당 지급지시 확인\n• 제수당 지급 심사위원회 개최 / 선발\n  - 인사명령 발령\n  - 국방통합급여정보체계 입력\n국군재정관리단• 통합급여정보체계 확인\n• 제수당 입금\n  - 매월 10일( 실적수당 25일)\n수당 입금확인( 개인)• 국방통합급여정보체계 확인\n  - 국방통합급여포탈 → 수당조회- 2 -      라.여단급 (준장)이상 부대의 인사참모는 필요시 분기 단위 심의를 실시한 후 그 결과를\n재정참모에게 통보한다. 여단 및 대대 인사과장은 수당 심의에 필요한 자료\n(명령지, 자격증 사본 등)를 국방인사정보체계를 이용하여 여단급(준장)이상 \n부대의 인사참모에게 제출하여야 한다.\n(수당 변동사항 발생 시 수시 심의 후 결과를 재정참모에게 통보하며 ,지급대상자 전출 /보직\n변경 시 전 소속부대에서 필수적으로 해임처리 )\n마.재정참모는 심의 결과 중 중앙지급수당과 관련 신규 임명 및 해임 등 변동사항을\n국군재정관리단( 급여연금처 수당관리과) 으로 제출한다.\n바.수당 인사명령 발령 시 수당담당은 다음 날 ‘급여체계 -제수당관리 -인사  \n명령자동처리결과 ’에서 처리 /반려 된 건을 반드시 확인하여 반려처리 된   \n건은 인사담당에게 재 발령을 의뢰해야 한다 .\n(국방인사정보체계 -국방통합급여정보체계 간 연동된 자료와 공문으로 발송된 인사\n명령지 상 내용이상이한 경우 국방인사정보체계 -국방

In [117]:
raw = [temp[1]]
raw
for i in range(len(raw)):
    # raw[i] = re.sub(r'제\s*(\d+)\s*(편|장|절|관)',r'제\1\2 ',raw[i])          # 공백 (숫자) 절 공백 제외 (예. 제 3 편)
    # raw[i] = re.sub(r'제\s*(\d+)\s*조\s*(\()\s*', r'제\1조\2', raw[i])          # '제 n 조 ('와 같은 항목 통일성 유지
    # raw[i] = re.sub(r'제\s*(\d+)\s*조\s*(\d+)\s*(\(|\<)',r'제\1조의\2\3',raw[i])     # '제n조의 n ('와 같은 항목 통일성 유지
    # raw[i] = re.sub(r'제\s*(\d+)\s*조의\s*(\d+)\s*(\(|\<)',r'제\1조의\2\3',raw[i])     # '제n조의 n ('와 같은 항목 통일성 유지
    # raw[i] = re.sub(r'제(\d+)조\s*(\d+)\s*(\(|\<)', r'제\1조의\2\3', raw[i])  # '제n조 n을 제n조의 n으로 통일
    # raw[i] = re.sub(r'제(\d+)조\s*삭\s*제', r'제\1조(삭제)', raw[i])           # '제n조 삭제를 제n조(삭제)로 통일
    # raw[i] = re.sub(r'제\s*(\d+)\s*조\s*<삭\s*제>', r'제\1조<삭제>', raw[i])           # '제n조 삭제를 제n조(삭제)로 통일
    # raw[i] = re.sub(r'제(\d+)조\s*(\d+)\s*삭\s*제', r'제\1조의\2(삭제)', raw[i])           # '제n조 삭제를 제n조(삭제)로 통일
    # raw[i] = re.sub(r'제(\d+)조의\s*(\d+)\s*삭\s*제', r'제\1조의\2(삭제)', raw[i])           # '제n조 삭제를 제n조(삭제)로 통일
    # raw[i] = re.sub(r'\s*부\s*칙\s*',r'부칙', raw[i])                        # 부칙 사이 공백 제거
    # raw[i] = re.sub(r'\t*부\t*칙\t*',r'부칙', raw[i])                        # 부칙 사이 공백 제거
    # raw[i] = re.sub(r'제(\d+)조\[(.*)\]',r'제\1조(\2)',raw[i])               # n조의 2와 같은 항목 통일성 유지
    raw[i] = re.sub(r'(\s)+', r'\1', raw[i])                                       # 공백이 여러개일 경우 한개만 둠
    raw[i] = re.sub(r'(\s)+\n', r'\1\n', raw[i])                                   # \n은 띄어쓰기로 변환
    raw[i] = re.sub(r'([^\.])\n', r'\1\n', raw[i])                              # 마침표가 아닌 경우 \n 유지
    raw[i] = re.sub(r'([^\.])\n', r'\1 ', raw[i])  # 마침표가 아닌 경우 \n을 공백으로 변환
    raw[i] = re.sub(r'(\.)(\s*\n)', r'\1\n', raw[i])  # 마침표가 있으면 \n 유지
    # raw[i] = re.sub('(\s*)\)',')',raw[i])                                   # ' )' 공백 제거
    # raw[i] = re.sub('\s*\·\s*',' ',raw[i])                                  # ·와 같은 특수문자는 나중에 규칙을 혼란스럽게 만들 수 있어 제외
    # raw[i] = re.sub('&#\d*;',' ',raw[i])                                    # 특수문자 제외
    # raw[i] = re.sub(' \(삭제\) ','(삭제)',raw[i])                            # 삭제조항 공백 제거
    # raw[i] = re.sub('-\s+(\d+)\s+-','',raw[i])                              # page numbering 삭제
    # raw[i] = re.sub('^\d+\s*-\s*\d+$','',raw[i])                            # 페이지 넘버링만 있는 리스트 삭제
    # raw[i] = re.sub(r'\s*작\s*성\s*(관|자)\s*:?\s*', r'작성관', raw[i])       # 작성관으로 통일
    # raw[i] = re.sub(r'\s*작\s*성\s*담\s*당\s*(관|자)\s*:?\s*', r'작성관', raw[i])       # 작성관으로 통일
    # raw[i] = re.sub(r'\s*작\s*성\s*책\s*임\s*(관|자)\s*:?\s*', r'작성관', raw[i])       # 작성관으로 통일
    # raw[i] = re.sub(r'국\s+방\s+부\s+장\s+관', r'', raw[i])                  # 이제 필요 없으니 삭제
    # raw[i] = re.sub(r'육\s+군\s+참\s+모\s+총\s+장', r'', raw[i])                  # 이제 필요 없으니 삭제
    # raw[i] = re.sub(r'(제\s+9\s+보\s+병\s+사\s+단(\s+장)?)', r'', raw[i])                  # 이제 필요 없으니 삭제
    # raw[i] = re.sub(r'조\((.+?)\((.+?)\)(.+?)\)',r'조(\1<\2>\3)',raw[i])    # 조 안에 중첩괄호 <>으로 바꾸기
    # raw[i] = re.sub(r'(\d*)\s*\.\s*(\d+)\s*\.\s*(\d+)\s*\.?', r'\1.\2.\3.', raw[i])  # nnnn.nn.nn. 으로 통일
    # raw[i] = re.sub(r'(\s*사단행정예규\s*\d+\s*\-\s*\d+)', r'', raw[i])            # 불필요한 텍스트 제거
    # raw[i] = re.sub(r'(\ž|\Ÿ)', r'-', raw[i])  # 불필요한 텍스트 제거
    # raw[i] = re.sub(r'(\-\s+(\d{1})\s+\-\s*)', r'', raw[i])              # 페이지 쪽 번호 제거

print(raw[0])
raw[0]

제2조(일반지침) 가.수당지급에 관한 사항은 본 지시에 의하고 ,임의로 지급대상자 선정기준 및 지급액 등을 조정할 수 없다.
나.수당의 지급방법 (중앙지급 ,부대지급 )과 정․결산에 관한 사항은 국군재정관리단장이 정한다 . 다. 제수당 신청절차 및 지급 수당신청( 개인)• 제수당 지급지침 확인 • 제수당 신청( 국방인사정보체계) 수당 / 행정담당 (부대(서))• 제수당 신청 접수 / 확인 • 제수당 지급지침 확인 • 제수당 지급 심사위원회 개최 / 선발 제수당 지급심사위원회 (사(여)단급 이상 부대)• 제수당 지급지시 확인 • 제수당 지급 심사위원회 개최 / 선발 - 인사명령 발령 - 국방통합급여정보체계 입력 국군재정관리단• 통합급여정보체계 확인 • 제수당 입금 - 매월 10일( 실적수당 25일) 수당 입금확인( 개인)• 국방통합급여정보체계 확인 - 국방통합급여포탈 → 수당조회- 2 - 라.여단급 (준장)이상 부대의 인사참모는 필요시 분기 단위 심의를 실시한 후 그 결과를 재정참모에게 통보한다. 여단 및 대대 인사과장은 수당 심의에 필요한 자료 (명령지, 자격증 사본 등)를 국방인사정보체계를 이용하여 여단급(준장)이상 부대의 인사참모에게 제출하여야 한다.
(수당 변동사항 발생 시 수시 심의 후 결과를 재정참모에게 통보하며 ,지급대상자 전출 /보직 변경 시 전 소속부대에서 필수적으로 해임처리 ) 마.재정참모는 심의 결과 중 중앙지급수당과 관련 신규 임명 및 해임 등 변동사항을 국군재정관리단( 급여연금처 수당관리과) 으로 제출한다.
바.수당 인사명령 발령 시 수당담당은 다음 날 ‘급여체계 -제수당관리 -인사 명령자동처리결과 ’에서 처리 /반려 된 건을 반드시 확인하여 반려처리 된 건은 인사담당에게 재 발령을 의뢰해야 한다 .
(국방인사정보체계 -국방통합급여정보체계 간 연동된 자료와 공문으로 발송된 인사 명령지 상 내용이상이한 경우 국방인사정보체계 -국방통합급여정보체계 간 자동 연동된 자료 내용을 반영한다 .) 사. 부대별 제수당 지급심사위원회 1) 심사위원회는 지

'제2조(일반지침) 가.수당지급에 관한 사항은 본 지시에 의하고 ,임의로 지급대상자 선정기준 및 지급액 등을 조정할 수 없다.\n나.수당의 지급방법 (중앙지급 ,부대지급 )과 정․결산에 관한 사항은 국군재정관리단장이 정한다 . 다. 제수당 신청절차 및 지급 수당신청( 개인)• 제수당 지급지침 확인 • 제수당 신청( 국방인사정보체계) 수당 / 행정담당 (부대(서))• 제수당 신청 접수 / 확인 • 제수당 지급지침 확인 • 제수당 지급 심사위원회 개최 / 선발 제수당 지급심사위원회 (사(여)단급 이상 부대)• 제수당 지급지시 확인 • 제수당 지급 심사위원회 개최 / 선발 - 인사명령 발령 - 국방통합급여정보체계 입력 국군재정관리단• 통합급여정보체계 확인 • 제수당 입금 - 매월 10일( 실적수당 25일) 수당 입금확인( 개인)• 국방통합급여정보체계 확인 - 국방통합급여포탈 → 수당조회- 2 - 라.여단급 (준장)이상 부대의 인사참모는 필요시 분기 단위 심의를 실시한 후 그 결과를 재정참모에게 통보한다. 여단 및 대대 인사과장은 수당 심의에 필요한 자료 (명령지, 자격증 사본 등)를 국방인사정보체계를 이용하여 여단급(준장)이상 부대의 인사참모에게 제출하여야 한다.\n(수당 변동사항 발생 시 수시 심의 후 결과를 재정참모에게 통보하며 ,지급대상자 전출 /보직 변경 시 전 소속부대에서 필수적으로 해임처리 ) 마.재정참모는 심의 결과 중 중앙지급수당과 관련 신규 임명 및 해임 등 변동사항을 국군재정관리단( 급여연금처 수당관리과) 으로 제출한다.\n바.수당 인사명령 발령 시 수당담당은 다음 날 ‘급여체계 -제수당관리 -인사 명령자동처리결과 ’에서 처리 /반려 된 건을 반드시 확인하여 반려처리 된 건은 인사담당에게 재 발령을 의뢰해야 한다 .\n(국방인사정보체계 -국방통합급여정보체계 간 연동된 자료와 공문으로 발송된 인사 명령지 상 내용이상이한 경우 국방인사정보체계 -국방통합급여정보체계 간 자동 연동된 자료 내용을 반영한다 .) 사. 부대별 제수당 지급심사위원회 1) 심사위

In [116]:
raw

['제2조(일반지침) 가.수당지급에 관한 사항은 본 지시에 의하고 ,임의로 지급대상자 선정기준 및 지급액 등을 조정할 수 없다. 나.수당의 지급방법 (중앙지급 ,부대지급 )과 정․결산에 관한 사항은 국군재정관리단장이 정한다 . 다. 제수당 신청절차 및 지급 수당신청( 개인)• 제수당 지급지침 확인 • 제수당 신청( 국방인사정보체계) 수당 / 행정담당 (부대(서))• 제수당 신청 접수 / 확인 • 제수당 지급지침 확인 • 제수당 지급 심사위원회 개최 / 선발 제수당 지급심사위원회 (사(여)단급 이상 부대)• 제수당 지급지시 확인 • 제수당 지급 심사위원회 개최 / 선발 - 인사명령 발령 - 국방통합급여정보체계 입력 국군재정관리단• 통합급여정보체계 확인 • 제수당 입금 - 매월 10일( 실적수당 25일) 수당 입금확인( 개인)• 국방통합급여정보체계 확인 - 국방통합급여포탈 → 수당조회- 2 - 라.여단급 (준장)이상 부대의 인사참모는 필요시 분기 단위 심의를 실시한 후 그 결과를 재정참모에게 통보한다. 여단 및 대대 인사과장은 수당 심의에 필요한 자료 (명령지, 자격증 사본 등)를 국방인사정보체계를 이용하여 여단급(준장)이상 부대의 인사참모에게 제출하여야 한다. (수당 변동사항 발생 시 수시 심의 후 결과를 재정참모에게 통보하며 ,지급대상자 전출 /보직 변경 시 전 소속부대에서 필수적으로 해임처리 ) 마.재정참모는 심의 결과 중 중앙지급수당과 관련 신규 임명 및 해임 등 변동사항을 국군재정관리단( 급여연금처 수당관리과) 으로 제출한다. 바.수당 인사명령 발령 시 수당담당은 다음 날 ‘급여체계 -제수당관리 -인사 명령자동처리결과 ’에서 처리 /반려 된 건을 반드시 확인하여 반려처리 된 건은 인사담당에게 재 발령을 의뢰해야 한다 . (국방인사정보체계 -국방통합급여정보체계 간 연동된 자료와 공문으로 발송된 인사 명령지 상 내용이상이한 경우 국방인사정보체계 -국방통합급여정보체계 간 자동 연동된 자료 내용을 반영한다 .) 사. 부대별 제수당 지급심사위원회 1) 심사위원회는

In [102]:
raw = [temp[1]]
print(re.sub(r'([^\.])\n',r'\1 ',raw[0]))

제2조(일반지침) 가.수당지급에 관한 사항은 본 지시에 의하고 ,임의로 지급대상자 선정기준 및 지급액 등을 조정할 수 없다.
나.수당의 지급방법 (중앙지급 ,부대지급 )과 정․결산에 관한 사항은 국군재정관리단장이 정한다 .
       다. 제수당 신청절차 및 지급 수당신청( 개인)• 제수당 지급지침 확인 • 제수당 신청( 국방인사정보체계) 수당 / 행정담당 (부대(서))• 제수당 신청 접수 / 확인 • 제수당 지급지침 확인 • 제수당 지급 심사위원회 개최 / 선발 제수당  지급심사위원회 (사(여)단급 이상 부대)• 제수당 지급지시 확인 • 제수당 지급 심사위원회 개최 / 선발   - 인사명령 발령   - 국방통합급여정보체계 입력 국군재정관리단• 통합급여정보체계 확인 • 제수당 입금   - 매월 10일( 실적수당 25일) 수당 입금확인( 개인)• 국방통합급여정보체계 확인   - 국방통합급여포탈 → 수당조회- 2 -      라.여단급 (준장)이상 부대의 인사참모는 필요시 분기 단위 심의를 실시한 후 그 결과를 재정참모에게 통보한다. 여단 및 대대 인사과장은 수당 심의에 필요한 자료 (명령지, 자격증 사본 등)를 국방인사정보체계를 이용하여 여단급(준장)이상  부대의 인사참모에게 제출하여야 한다.
(수당 변동사항 발생 시 수시 심의 후 결과를 재정참모에게 통보하며 ,지급대상자 전출 /보직 변경 시 전 소속부대에서 필수적으로 해임처리 ) 마.재정참모는 심의 결과 중 중앙지급수당과 관련 신규 임명 및 해임 등 변동사항을 국군재정관리단( 급여연금처 수당관리과) 으로 제출한다.
바.수당 인사명령 발령 시 수당담당은 다음 날 ‘급여체계 -제수당관리 -인사   명령자동처리결과 ’에서 처리 /반려 된 건을 반드시 확인하여 반려처리 된    건은 인사담당에게 재 발령을 의뢰해야 한다 .
(국방인사정보체계 -국방통합급여정보체계 간 연동된 자료와 공문으로 발송된 인사 명령지 상 내용이상이한 경우 국방인사정보체계 -국방통합급여정보체계 간 자동 연동된 자료 내용을 반영한다 .) 사. 

In [23]:
print(re.sub('(\s*)\n',' ',raw[0]))

제1조(목  적) 이 지시는 「공무원 수당 등에 관한 규정」(이하 “영”이라 한다)「군인 등의 특수근무 수당에 관한 규칙」(이하 “규칙”이라 한다)및「공무원 보수 등의 업무지침 」등에 의하여 군인 및 군무원에게 적용하는 제수당의 지급기준과 세부절차를 정함을 목적으로 한다. 


In [ ]:
print(re.sub('(\s*)\n',' ',raw[0]))

In [32]:
# 불필요한 값들 삭제
for i in range(len(raw)):
    raw[i] = re.sub(r'제\s*(\d+)\s*(편|장|절|관)',r'제 \1\2 ',raw[i])        # 공백 (숫자) 절 공백 제외 (예. 제 3 편)
    raw[i] = re.sub(r'제(\d+)조의 (\d+)',r'제\1조의\2',raw[i])            # n조의 2와 같은 항목 통일성 유지
    raw[i] = re.sub(r'부(\s)*칙(\s)*',r'부칙',raw[i])                     # 부칙 사이 공백 제거
    raw[i] = re.sub(r'제(\d+)조\[(.*)\]',r'제\1조(\2)',raw[i])            # 대괄호를 소괄호로 변경
    raw[i] = re.sub(r'제(\d+)조\(\s+',r'제\1조(',raw[i])  
    raw[i] = re.sub('\s+',' ',raw[i])                                    # 공백이 여러개일 경우 한개만 둠
    raw[i] = re.sub('(\s)*\n',' ',raw[i])                                # \n은 띄어쓰기로 변환
    raw[i] = re.sub('(\s)*\)',')',raw[i])                                # \n은 띄어쓰기로 변환
    raw[i] = re.sub('·',' ',raw[i])                                      # ·와 같은 특수문자는 나중에 규칙을 혼란스럽게 만들 수 있어 제외
    raw[i] = re.sub('&#\d*;',' ',raw[i])                                 # 특수문자 제외
    raw[i] = re.sub(' \(삭제\) ','(삭제)',raw[i])                         # 삭제조항 공백 제거
    raw[i] = re.sub('-\s+(\d+)\s+-','',raw[i])                           # page numbering 삭제
    raw[i] = re.sub('^\d+-\d+$','',raw[i])                               # 페이지 넘버링만 있는 리스트 삭제
    raw[i] = re.sub(r'\s*작\s*성\s*관', r'작성관', raw[i])                # 작성관으로 변경
    raw[i] = re.sub(r'국\s*방\s*부\s*장\s*관', r'', raw[i])               # 이제 필요 없으니 삭제
    raw[i] = re.sub(r'조\((.+?)\((.+?)\)(.+?)\)',r'조(\1<\2>\3)',raw[i])  # 조 안에 중첩괄호 <>으로 바꾸기

raw = [txt for txt in raw if txt] ## 빈 문자열 제거

In [22]:
raw

['제1조(목 적) 이 지시는 「공무원 수당 등에 관한 규정」<이하 “영”이라 한다>「군인 등의 특수근무 수당에 관한 규칙」(이하 “규칙”이라 한다)및「공무원 보수 등의 업무지침 」등에 의하여 군인 및 군무원에게 적용하는 제수당의 지급기준과 세부절차를 정함을 목적으로 한다. ',
 '제2조(일반지침) 가.수당지급에 관한 사항은 본 지시에 의하고 ,임의로 지급대상자 선정기준 및 지급액 등을 조정할 수 없다. 나.수당의 지급방법 <중앙지급 ,부대지급>과 정․결산에 관한 사항은 국군재정관리단장이 정한다 . 다. 제수당 신청절차 및 지급 수당신청( 개인)• 제수당 지급지침 확인 • 제수당 신청( 국방인사정보체계) 수당 / 행정담당 (부대(서))• 제수당 신청 접수 / 확인 • 제수당 지급지침 확인 • 제수당 지급 심사위원회 개최 / 선발 제수당 지급심사위원회 (사(여)단급 이상 부대)• 제수당 지급지시 확인 • 제수당 지급 심사위원회 개최 / 선발 - 인사명령 발령 - 국방통합급여정보체계 입력 국군재정관리단• 통합급여정보체계 확인 • 제수당 입금 - 매월 10일( 실적수당 25일) 수당 입금확인( 개인)• 국방통합급여정보체계 확인 - 국방통합급여포탈 → 수당조회 라.여단급 (준장)이상 부대의 인사참모는 필요시 분기 단위 심의를 실시한 후 그 결과를 재정참모에게 통보한다. 여단 및 대대 인사과장은 수당 심의에 필요한 자료 (명령지, 자격증 사본 등)를 국방인사정보체계를 이용하여 여단급(준장)이상 부대의 인사참모에게 제출하여야 한다. (수당 변동사항 발생 시 수시 심의 후 결과를 재정참모에게 통보하며 ,지급대상자 전출 /보직 변경 시 전 소속부대에서 필수적으로 해임처리) 마.재정참모는 심의 결과 중 중앙지급수당과 관련 신규 임명 및 해임 등 변동사항을 국군재정관리단( 급여연금처 수당관리과) 으로 제출한다. 바.수당 인사명령 발령 시 수당담당은 다음 날 ‘급여체계 -제수당관리 -인사 명령자동처리결과 ’에서 처리 /반려 된 건을 반드시 확인하여 반려처리 된 건은 인사담당에게

In [23]:
# pattern = '(제 [0-9]{1,3} 편)+(\s*\w*)*(제 [0-9]{1,3} 장)*(\s*\w*)*(제[0-9]{1,3}절)*(\s*\w*)*(제[0-9]{1,3}조)+\((\s*\w*)+\)'

pattern = r'제[0-9]{1,3}조(의\s*\d+\s*)?\s*\((.+?)\)|(부칙)|(작성관)'
name_com = re.compile(pattern)

# candidate pool jsonl 생성
candidate_pool = []
for i, txt in tqdm(enumerate(raw)):
    try:
        split_std = name_com.search(txt).span()[1]  # 이름과 제목 끊을 라인
        txt_dic = {
            'subject':txt[:split_std],
            'text':txt[split_std:],
            'reference': txt_file_name.replace('.txt','')
            }
        candidate_pool.append(txt_dic)
    except AttributeError:
        print(str(i)+'행의"'+ txt + '" 문장을 확인해주세요.')
        pass

50it [00:00, 46020.45it/s]


In [24]:
candidate_pool

[{'subject': '제1조(목 적)',
  'text': ' 이 지시는 「공무원 수당 등에 관한 규정」<이하 “영”이라 한다>「군인 등의 특수근무 수당에 관한 규칙」(이하 “규칙”이라 한다)및「공무원 보수 등의 업무지침 」등에 의하여 군인 및 군무원에게 적용하는 제수당의 지급기준과 세부절차를 정함을 목적으로 한다. ',
  'reference': '22-1012'},
 {'subject': '제2조(일반지침)',
  'text': ' 가.수당지급에 관한 사항은 본 지시에 의하고 ,임의로 지급대상자 선정기준 및 지급액 등을 조정할 수 없다. 나.수당의 지급방법 <중앙지급 ,부대지급>과 정․결산에 관한 사항은 국군재정관리단장이 정한다 . 다. 제수당 신청절차 및 지급 수당신청( 개인)• 제수당 지급지침 확인 • 제수당 신청( 국방인사정보체계) 수당 / 행정담당 (부대(서))• 제수당 신청 접수 / 확인 • 제수당 지급지침 확인 • 제수당 지급 심사위원회 개최 / 선발 제수당 지급심사위원회 (사(여)단급 이상 부대)• 제수당 지급지시 확인 • 제수당 지급 심사위원회 개최 / 선발 - 인사명령 발령 - 국방통합급여정보체계 입력 국군재정관리단• 통합급여정보체계 확인 • 제수당 입금 - 매월 10일( 실적수당 25일) 수당 입금확인( 개인)• 국방통합급여정보체계 확인 - 국방통합급여포탈 → 수당조회 라.여단급 (준장)이상 부대의 인사참모는 필요시 분기 단위 심의를 실시한 후 그 결과를 재정참모에게 통보한다. 여단 및 대대 인사과장은 수당 심의에 필요한 자료 (명령지, 자격증 사본 등)를 국방인사정보체계를 이용하여 여단급(준장)이상 부대의 인사참모에게 제출하여야 한다. (수당 변동사항 발생 시 수시 심의 후 결과를 재정참모에게 통보하며 ,지급대상자 전출 /보직 변경 시 전 소속부대에서 필수적으로 해임처리) 마.재정참모는 심의 결과 중 중앙지급수당과 관련 신규 임명 및 해임 등 변동사항을 국군재정관리단( 급여연금처 수당관리과) 으로 제출한다. 바.수당 인사명령 발령 시

In [25]:
# candidate pool jsonl 형태로 저장
with open('./candidate_pool/'+ txt_file_name.replace('.txt','.jsonl'),encoding='utf-8', mode="w") as file:
    for i in candidate_pool:
        file.write(json.dumps(i, ensure_ascii=False, indent = 3) + "\n")

PermissionError: [Errno 13] Permission denied: './candidate_pool/22-1012.jsonl'

In [ ]:
# candidate pool csv 형태로 저장
import pandas as pd
pd.DataFrame(candidate_pool).to_csv('./candidate_pool/'+file_name+'.csv', encoding='euc-kr')

In [ ]:
import jsonlines

# with jsonlines.open('./candidate_pool/'+ file_name + '.jsonl') as f:
#     for line in f.iter():
#         print(line["id"]) # 각 json에 해당하는 "id" 출력
        # print(line["title"]) # 각 json에 해당하는 "title" 출력

In [ ]:
f = jsonlines.open('./candidate_pool/'+ file_name + '.jsonl')

In [ ]:
# 확인 
# with open('./candidate_pool/candidate_pool'+ file_name + ".jsonl") as f: 
# 	for line in f: print(line)